<a href="https://colab.research.google.com/github/sproboticworks/ml-course/blob/master/Cats%20and%20Dogs%20Classification%20using%20Transfer%20Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Packages

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras import layers


#Download Data

In [0]:
url = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
zip_dir = tf.keras.utils.get_file('cats_and_dogs_filtered.zip', origin=url, extract=True)

In [0]:
base_dir = os.path.join(os.path.dirname(zip_dir), 'cats_and_dogs_filtered')

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat/dog pictures
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our validation cat/dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

num_cats_tr = len(os.listdir(train_cats_dir))
num_dogs_tr = len(os.listdir(train_dogs_dir))

num_cats_val = len(os.listdir(validation_cats_dir))
num_dogs_val = len(os.listdir(validation_dogs_dir))

total_train = num_cats_tr + num_dogs_tr
total_val = num_cats_val + num_dogs_val

# Preprocess Data

In [0]:
BATCH_SIZE = 20
IMG_SHAPE = 224
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
validation_datagen  = ImageDataGenerator( rescale = 1.0/255. )

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='binary',
                                                    target_size=(IMG_SHAPE, IMG_SHAPE))
     
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  validation_datagen.flow_from_directory(validation_dir,
                                                         batch_size=BATCH_SIZE,
                                                         class_mode  = 'binary',
                                                         target_size = (IMG_SHAPE, IMG_SHAPE))

# Load Pre-trained model

In [0]:
import tensorflow_hub as hub

CLASSIFIER_URL ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"
IMAGE_RES = 224

feature_extractor = hub.KerasLayer(CLASSIFIER_URL, input_shape=(IMAGE_RES, IMAGE_RES,3))
feature_extractor.trainable = False

In [0]:
model = tf.keras.Sequential([
    feature_extractor,
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Train Model

In [0]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics = ['accuracy'])

In [0]:
EPOCHS = 5
history = model.fit(train_generator,
                              validation_data=validation_generator,
                              steps_per_epoch=int(np.ceil(total_train / float(BATCH_SIZE))),
                              epochs=EPOCHS,
                              validation_steps=int(np.ceil(total_val / float(BATCH_SIZE))))

# Prediction using the model

In [0]:
test_images, test_labels = next(validation_generator)
classes = model.predict(test_images, 10)
classes = classes.flatten()
fig, axes = plt.subplots(4, 5, figsize=(10,10))
axes = axes.flatten()
i = 0
for img, ax in zip(test_images, axes):
    ax.imshow(img)
    ax.axis('off')
    color = 'blue'
    if round(classes[i]) != test_labels[i] :
        color = 'red'
    if classes[i]>0.5:
        ax.set_title("Dog",fontdict = {'size' : 20, 'color' : color});
    else :
        ax.set_title("Cat",fontdict = {'size' : 20, 'color' : color});
    i+=1
plt.tight_layout()
plt.show()

# Save as Keras .h5 model

In [0]:
import time
t = time.time()

export_path_keras = "./{}.h5".format(int(t))
print(export_path_keras)
model.save(export_path_keras)

# Load the Keras .h5 Model

In [0]:
reloaded = tf.keras.models.load_model(
  export_path_keras, 
  # `custom_objects` tells keras how to load a `hub.KerasLayer`
  custom_objects={'KerasLayer': hub.KerasLayer})

reloaded.summary()

In [0]:
result_batch = model.predict(test_images)
reloaded_result_batch = reloaded.predict(test_images)
(abs(result_batch - reloaded_result_batch)).max()

## Keep Training

In [0]:
EPOCHS = 3
history = reloaded.fit(train_generator,
                    epochs=EPOCHS,
                    validation_data=validation_generator)

# Export as SavedModel

In [0]:
t = time.time()

export_path_sm = "./{}".format(int(t))
print(export_path_sm)

tf.saved_model.save(model, export_path_sm)

# Load SavedModel

In [0]:
reloaded_sm = tf.saved_model.load(export_path_sm)
reload_sm_result_batch = reloaded_sm(test_images, training=False).numpy()
(abs(result_batch - reload_sm_result_batch)).max()

# Loading the SavedModel as a Keras Model

In [0]:
t = time.time()

export_path_sm = "./{}".format(int(t))
print(export_path_sm)
tf.saved_model.save(model, export_path_sm)

In [0]:
reload_sm_keras = tf.keras.models.load_model(
  export_path_sm,
  custom_objects={'KerasLayer': hub.KerasLayer})

reload_sm_keras.summary()

In [0]:
result_batch = model.predict(test_images)
reload_sm_keras_result_batch = reload_sm_keras.predict(test_images)

In [0]:
(abs(result_batch - reload_sm_keras_result_batch)).max()